In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d saurabhshahane/fake-news-classification

fake-news-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
zipfile_ref = zipfile.ZipFile("/content/fake-news-classification.zip",'r')
zipfile_ref.extractall("/content")
zipfile_ref.close()

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
dataset = pd.read_csv("/content/WELFake_Dataset.csv")

In [8]:
dataset.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [9]:
dataset.shape

(72134, 4)

In [10]:
dataset.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [11]:
dataset = dataset.fillna("")

In [12]:
dataset.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [13]:
X = dataset.drop(columns='label', axis=1)
Y = dataset['label']

In [14]:
print(X)
print(Y)

       Unnamed: 0                                              title  \
0               0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1               1                                                      
2               2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3               3  Bobby Jindal, raised Hindu, uses story of Chri...   
4               4  SATAN 2: Russia unvelis an image of its terrif...   
...           ...                                                ...   
72129       72129  Russians steal research on Trump in hack of U....   
72130       72130   WATCH: Giuliani Demands That Democrats Apolog...   
72131       72131  Migrants Refuse To Leave Train At Refugee Camp...   
72132       72132  Trump tussle gives unpopular Mexican leader mu...   
72133       72133  Goldman Sachs Endorses Hillary Clinton For Pre...   

                                                    text  
0      No comment is expected from Barack Obama Membe...  
1         Did the

In [15]:
dataset['content'] = dataset['title']+''+dataset['text']

In [16]:
print(dataset['content'])

0        LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1           Did they post their votes for Hillary already?
2        UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3        Bobby Jindal, raised Hindu, uses story of Chri...
4        SATAN 2: Russia unvelis an image of its terrif...
                               ...                        
72129    Russians steal research on Trump in hack of U....
72130     WATCH: Giuliani Demands That Democrats Apolog...
72131    Migrants Refuse To Leave Train At Refugee Camp...
72132    Trump tussle gives unpopular Mexican leader mu...
72133    Goldman Sachs Endorses Hillary Clinton For Pre...
Name: content, Length: 72134, dtype: object


In [17]:
# port_stem = PorterStemmer()

In [18]:
# # error maaybe present here
# def stemming(content):
#   stemmed_content = re.sub('[^a-zA-Z]','', content)
#   stemmed_content = stemmed_content.lower()
#   stemmed_content = stemmed_content.split()
#   stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
#   stemmed_content = ''.join(stemmed_content)
#   return stemmed_content

In [19]:
# dataset['content'] = dataset['content'].apply(stemming)

In [20]:
print(dataset['content'])

0        LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1           Did they post their votes for Hillary already?
2        UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3        Bobby Jindal, raised Hindu, uses story of Chri...
4        SATAN 2: Russia unvelis an image of its terrif...
                               ...                        
72129    Russians steal research on Trump in hack of U....
72130     WATCH: Giuliani Demands That Democrats Apolog...
72131    Migrants Refuse To Leave Train At Refugee Camp...
72132    Trump tussle gives unpopular Mexican leader mu...
72133    Goldman Sachs Endorses Hillary Clinton For Pre...
Name: content, Length: 72134, dtype: object


In [21]:
X = dataset['content'].values
Y = dataset['label'].values

In [22]:
print(X)

['LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:3

In [23]:
print(Y)

[1 1 1 ... 0 0 1]


In [24]:
Y.shape

(72134,)

In [25]:
X.shape

(72134,)

In [26]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [27]:
print(X)

  (0, 245604)	0.02224944590381374
  (0, 245539)	0.028008116882150193
  (0, 245508)	0.018077122870620133
  (0, 245463)	0.019092683132846493
  (0, 245273)	0.11738008619729111
  (0, 244893)	0.03804410133660806
  (0, 242646)	0.01676774070498498
  (0, 241489)	0.012490433197262528
  (0, 241222)	0.038464052228729846
  (0, 240721)	0.07542821077720792
  (0, 240511)	0.01977722702353259
  (0, 240204)	0.014878951990629195
  (0, 240126)	0.028506413162765833
  (0, 240059)	0.11146335868484777
  (0, 239876)	0.011615255129603026
  (0, 239867)	0.05378636423674954
  (0, 239782)	0.02140999726844785
  (0, 239773)	0.01798105976669477
  (0, 239198)	0.012188999722213545
  (0, 238734)	0.029745467230579997
  (0, 238518)	0.10693991434347527
  (0, 238334)	0.011987595799306123
  (0, 238110)	0.056619525448983646
  (0, 237933)	0.033811389163212566
  (0, 237582)	0.029816788439999044
  :	:
  (72133, 29925)	0.031758613114116284
  (72133, 29789)	0.012513839958900761
  (72133, 29575)	0.07940329422269916
  (72133, 29344)	

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state=2)

In [29]:
model = LogisticRegression()

In [30]:
model.fit(X_train, Y_train)

LogisticRegression()

In [31]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [32]:
print(training_data_accuracy)

0.9637132410279515


In [33]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [34]:
print(testing_data_accuracy)

0.9480834546336729


In [35]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("the news is real")
else:
  print("the news is fake")



[1]
the news is fake


In [36]:
print(Y_test[0])

1


In [38]:
X_new = X_test[1]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("the news is real")
else:
  print("the news is fake")



[0]
the news is real


In [39]:
print(Y_test[1])

0
